# Purpose: 
Join all office predicted products for plot generation and apply rule of choosing only products released between 2010 and 2018

In [1]:
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 10.0 MB/s eta 0:00:0000:0100:01


In [1]:
import pandas as pd 
import os 
import json 
import time 
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
from datetime import datetime

0it [00:00, ?it/s]

In [2]:
# path to predicted files: 
path = os.path.join('/home/ec2-user/SageMaker/', 'preds_new')

In [3]:
def create_predicted_df(path_to_predicted_files, path_to_product_release_file): 
      """
    Create a DataFrame by combining multiple predicted files and joining it with the product release information.

    Args:
        path_to_predicted_files (str): The path to the directory containing predicted files.
        path_to_product_release_file (str): The path to the product release file.

    Returns:
        pandas.DataFrame: The combined DataFrame with joined product release information.
    """
    file_list = os.listdir(path_to_predicted_files)
    # Major DataFrame to append all the  files
    major_df = pd.DataFrame()
    for file in file_list: 
        print(file)
        print('--------')
        try:
            # read the processed file
            temp_df= pd.read_json(os.path.join(path_to_predicted_files, file))
            temp_df = temp_df[['review_id', 'review_date', 'rating', 'review_body', 'date_converted',
                               'year', 'language', 'zero_stars', 'rating_management_explicit',
                                'wrong_buying', 'disagreement_with_ratings', 'read_reviews']]
            major_df = major_df.append(temp_df, ignore_index = True)
        except: 
            print(f'{file} not appended')
            print('**********')
    # drop duplicate va;ues
    major_df_1 = major_df.drop_duplicates()
    # read the office release date file
    office_release_df =  pq.read_table(path_to_product_release_file)
    # Convert the PyArrow table to a Pandas DataFrame
    office_release_df = office_release_df.to_pandas()
    # join the dataframes in order to get product_id and corresponding min_date 
    office_release_df = major_df_1.merge(office_release_df, on ='review_id', how='inner')
    
    return office_release_df

def extract_year(full_date): 
     """
    Extract the year from a full date string.

    Args:
        full_date (str): The full date string in the format 'YYYY-MM-DD HH:MM:SS'.

    Returns:
        int: The extracted year as an integer.
    """
    date_object = datetime.strptime(full_date, '%Y-%m-%d %H:%M:%S')
    year = date_object.year
    return year


In [4]:
major_office_df = create_predicted_df(path,'office_product_release_date.parquet')

amazon_reviews_office_1_read_reviews_predicted.json
--------
amazon_reviews_office_0_read_reviews_predicted.json
--------
amazon_reviews_office_3_read_reviews_predicted.json
--------
amazon_reviews_office_8_read_reviews_predicted.json
--------
amazon_reviews_office_7_read_reviews_predicted.json
--------
amazon_reviews_office_4_read_reviews_predicted.json
--------
amazon_reviews_office_9_read_reviews_predicted.json
--------
amazon_reviews_office_6_read_reviews_predicted.json
--------
amazon_reviews_office_5_read_reviews_predicted.json
--------
amazon_reviews_office_2_read_reviews_predicted.json
--------


In [5]:
major_office_df.head(10)

,review_id,review_date,rating,review_body,date_converted,year,language,zero_stars,rating_management_explicit,wrong_buying,disagreement_with_ratings,read_reviews,product_id,min_date
0,R1S2PIZEIYGBV7,"Reviewed in the United States on October 8, 2019",5.0 out of 5 stars,Gorgeous Leather Bound Diary...Old World Quali...,2019-10-08 00:00:00,2019,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00
1,R1EMNPBW34DL6Q,"Reviewed in the United States on September 22,...",4.0 out of 5 stars,"Awesome little notebook,just wish it came with...",2018-09-22 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00
2,R1OORI57AQ0SZD,"Reviewed in the United States on March 21, 2018",5.0 out of 5 stars,Very great quality. It has such a charm to it ...,2018-03-21 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00
3,R1GPO495D2A8ZN,"Reviewed in the United States on March 7, 2018",5.0 out of 5 stars,amazing quality and great price,2018-03-07 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00
4,R3Q9P0PVL448ID,"Reviewed in the United States on February 19, ...",5.0 out of 5 stars,My husband really likes this journel.,2018-02-19 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00
5,R1DKTFKYXGKPV,"Reviewed in the United States on February 12, ...",5.0 out of 5 stars,Searched for a diary that could be refilled an...,2018-02-12 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00
6,R2PFNXGI0KBSZK,"Reviewed in the United States on October 7, 2017",4.0 out of 5 stars,My 9-year-old grandson chose this journal. He...,2017-10-07 00:00:00,2017,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00
7,R96CJHJGR32N,"Reviewed in the United States on August 1, 2017",2.0 out of 5 stars,The cover is some sort of fiber board printed ...,2017-08-01 00:00:00,2017,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00
8,R2ENKMUFSK7HGP,"Reviewed in the United States on June 14, 2017",5.0 out of 5 stars,"Cool journal, great addition to my slowly grow...",2017-06-14 00:00:00,2017,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00
9,R317DOVOZNHAQ8,"Reviewed in the United States on October 12, 2015",2.0 out of 5 stars,bad quality,2015-10-12 00:00:00,2015,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00


In [6]:
# extract year from the product release date: 
major_office_df['product_year']= major_office_df.min_date.progress_apply(lambda x: extract_year(x))
major_office_df.head(10)

  0%|          | 0/1774786 [00:00<?, ?it/s]

,review_id,review_date,rating,review_body,date_converted,year,language,zero_stars,rating_management_explicit,wrong_buying,disagreement_with_ratings,read_reviews,product_id,min_date,product_year
0,R1S2PIZEIYGBV7,"Reviewed in the United States on October 8, 2019",5.0 out of 5 stars,Gorgeous Leather Bound Diary...Old World Quali...,2019-10-08 00:00:00,2019,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
1,R1EMNPBW34DL6Q,"Reviewed in the United States on September 22,...",4.0 out of 5 stars,"Awesome little notebook,just wish it came with...",2018-09-22 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
2,R1OORI57AQ0SZD,"Reviewed in the United States on March 21, 2018",5.0 out of 5 stars,Very great quality. It has such a charm to it ...,2018-03-21 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
3,R1GPO495D2A8ZN,"Reviewed in the United States on March 7, 2018",5.0 out of 5 stars,amazing quality and great price,2018-03-07 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
4,R3Q9P0PVL448ID,"Reviewed in the United States on February 19, ...",5.0 out of 5 stars,My husband really likes this journel.,2018-02-19 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
5,R1DKTFKYXGKPV,"Reviewed in the United States on February 12, ...",5.0 out of 5 stars,Searched for a diary that could be refilled an...,2018-02-12 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
6,R2PFNXGI0KBSZK,"Reviewed in the United States on October 7, 2017",4.0 out of 5 stars,My 9-year-old grandson chose this journal. He...,2017-10-07 00:00:00,2017,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
7,R96CJHJGR32N,"Reviewed in the United States on August 1, 2017",2.0 out of 5 stars,The cover is some sort of fiber board printed ...,2017-08-01 00:00:00,2017,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
8,R2ENKMUFSK7HGP,"Reviewed in the United States on June 14, 2017",5.0 out of 5 stars,"Cool journal, great addition to my slowly grow...",2017-06-14 00:00:00,2017,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
9,R317DOVOZNHAQ8,"Reviewed in the United States on October 12, 2015",2.0 out of 5 stars,bad quality,2015-10-12 00:00:00,2015,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014


In [7]:
# value_counts of product releases: 
prod_release_df=major_office_df[['product_id', 'product_year']].drop_duplicates()
prod_release_df.product_year.value_counts()

2018    3174
2016    2699
2017    2604
2015    2395
2014    2319
2013    1888
2012    1466
2019    1080
2011    1033
2010     778
2009     446
2008     304
2007     250
2006      88
2020      87
2005      52
2021      35
2004      32
2000      22
2003      15
2001      14
2002      11
2022       8
2023       2
1999       2
1998       1
Name: product_year, dtype: int64

In [8]:
# considering only products between 2010-2018: 
# Filter the DataFrame for years between 2010 and 2018 (inclusive)
major_df_2010_2018 = major_office_df[(major_office_df['product_year'] >= 2010) & (major_office_df['product_year'] <= 2018)]
major_df_2010_2018

,review_id,review_date,rating,review_body,date_converted,year,language,zero_stars,rating_management_explicit,wrong_buying,disagreement_with_ratings,read_reviews,product_id,min_date,product_year
0,R1S2PIZEIYGBV7,"Reviewed in the United States on October 8, 2019",5.0 out of 5 stars,Gorgeous Leather Bound Diary...Old World Quali...,2019-10-08 00:00:00,2019,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
1,R1EMNPBW34DL6Q,"Reviewed in the United States on September 22,...",4.0 out of 5 stars,"Awesome little notebook,just wish it came with...",2018-09-22 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
2,R1OORI57AQ0SZD,"Reviewed in the United States on March 21, 2018",5.0 out of 5 stars,Very great quality. It has such a charm to it ...,2018-03-21 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
3,R1GPO495D2A8ZN,"Reviewed in the United States on March 7, 2018",5.0 out of 5 stars,amazing quality and great price,2018-03-07 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
4,R3Q9P0PVL448ID,"Reviewed in the United States on February 19, ...",5.0 out of 5 stars,My husband really likes this journel.,2018-02-19 00:00:00,2018,en,0,0,0,0,0,B00OR7MNDY,2014-12-29 00:00:00,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1774781,RZKU48MX8QHPB,Reviewed in the United States 🇺🇸 on December 1...,5.0 out of 5 stars,Really good! Really like the quality of paper ...,2019-12-16 00:00:00,2019,en,0,0,0,0,0,B000Z9MR6G,2013-09-23 00:00:00,2013
1774782,R1FSC27RWUX8KH,"Reviewed in the United States 🇺🇸 on May 24, 2018",1.0 out of 5 stars,Sheesh! The paper in this notebook is THIN. Yo...,2018-05-24 00:00:00,2018,en,0,0,0,0,0,B000Z9MR6G,2013-09-23 00:00:00,2013
1774783,R20VYRIT885PVH,"Reviewed in the United States 🇺🇸 on August 11,...",5.0 out of 5 stars,Great Tablet ! 2nd purchase,2016-08-11 00:00:00,2016,en,0,0,0,0,0,B000Z9MR6G,2013-09-23 00:00:00,2013
1774784,R3ATA6MSYJ5MYR,"Reviewed in the United States 🇺🇸 on January 9,...",3.0 out of 5 stars,"While the description is accurate, the item is...",2014-01-09 00:00:00,2014,en,0,0,0,0,0,B000Z9MR6G,2013-09-23 00:00:00,2013


In [9]:
# Create a PyArrow table
table = pa.Table.from_pandas(major_df_2010_2018)

# Write the PyArrow table to a Parquet file
pq.write_table(table, 'office_product_analysis_file_rr.parquet')

In [30]:
major_df[major_df.zero_stars==1].review_body.iloc[100]

'I was very hopeful of this item helping make my life simple in labeling day care clothing. I followed every instruction to the last detail. But after a single wash, the labels completely came off and were found in the drier. I used the delicates cycle as this was for my baby. Extremely disappointed. Wish I could have given it 0 stars. Have contacted seller for refund.'